# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Terminal command to import the dataset: `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

Inline command to import the dataset: `!mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`